## Overview

In this project, you will practice applying your mechanistic simulation skills to create models that generate datasets you can analyze and optimize. 

## Global imports

Write your imports here so you don't have to write imports below.

In [1]:
!pip install simpy

In [17]:
import simpy as sp
import numpy as np
from typing import List, Tuple
import random
from typing import Callable
import scipy
from scipy.optimize import minimize_scalar

## Problem 1

**Write and discuss the steps to answering the following research question:** We would like to design a [network packet](https://en.wikipedia.org/wiki/Network_packet) routing system. To do so, we will need to define the system and run a simulation so that we can generate data that will be used for optimization of design parameters. Our router will be a bit of a simpler system, so do not worry about any specifics you know about ethernet or telecommunications in general.  The goal of our router is that it will route packets from devices connected to it through to the internet.

We'll parameterize our router so that it can accept $\alpha$ packet routing connections at any time. Assume that packets take $\beta$ time to be routed through the router. Let's say that each device connected to the router is only willing to wait up to $\gamma$ time for the packet to be routed to its destination.  Let's also say that our system will allow for prioritization on three levels, so that we can make sure those high-priority packets like those for voice and video calls can route through before lower-priority packets like standard file downloads.  To model our system, let's say that we have $n$ devices on our network connected to the router.  Each device generates $X_v$ videocalling packets, $X_s$ standard priority packets, and $X_l$ low priority packets at an $X_i$ interval.  Higher priority packets will not preempt lower priority packets, but they will be able to jump the queue (`simpy.PriorityResource`).

Based on the distributions for the parameters below, what is the average number of packets that make it through the router to the internet, for $n$ = [3, 5, 10, 15]?  All time-related parameters below are in milliseconds. Make sure to set the random seed.
- $\alpha$ = 6
- $\beta$ = exponentially-modified normal continuous random variable at location 1 and scale 1.5
- $\gamma$ = 3
- $X_v$ = uniform discrete random variable between [2,10]
- $X_s$ = uniform discrete random variable between [2,10]
- $X_l$ = uniform discrete random variable between [2,10]
- $X_i$ = binomial discrete random variable with n at 10 and p at 0.5

In [18]:
RANDOM_SEED = 5

alpha = 6

gamma = 3

min_beta = 1

max_beta = 1.5

def source(env, number, interval, counter, the_rng):
    """Source generates packets randomly"""
    for i in range(number):
        p = packet(env, 'Packets %02d' % i, counter, 12.0, the_rng)
        env.process(p)
        t = the_rng.exponential(interval)
        yield env.timeout(t)


def packet(env, name, counter, time_in_bank, the_rng):
    """Packet arrives, is served and leaves."""
    packet_list=['Xv','Xs','Xl','Xi']
    router_list=['ethernet','telecommunications']
    random_router = random.choice(router_list)
    random_packet = random.choice(packet_list)
    arrive = env.now
    print(random_packet+' at time %7.2f, %s arrived' % (arrive, name))
    
    with counter.request() as req:
        router = the_rng.uniform(min_beta, max_beta)
        # Wait for the counter or abort at the end of our tether
        results = yield req | env.timeout(router)

        wait = env.now - arrive

        if req in results:
            # We got to the counter
            print('at time %7.2f, %s waited %6.2f' % (env.now, name, wait))

            tib = the_rng.exponential(time_in_bank)
            yield env.timeout(tib)
            print('at time %7.2f, %s finished their rouing at router device %random_router' % (env.now, name, random_router))

        else:
            # We reneged
            print(' at time %7.2f, %s left the line after waiting %6.2f minutes' % (env.now, name, wait))


# Setup and start the simulation
print('SIMULATION:  average number of packets that make it through the router to the internet (time units are minutes)')
the_rng = np.random.default_rng(RANDOM_SEED)
env = sp.Environment()

# Start processes and run
counter = sp.Resource(env, capacity=1)
env.process(source(env, alpha, gamma, counter, the_rng))
env.run()

SIMULATION:  average number of packets that make it through the router to the internet (time units are minutes)
Xv at time    0.00, Packets 00 arrived
at time    0.00, Packets 00 waited   0.00
Xi at time    5.96, Packets 01 arrived
 at time    6.99, Packets 01 left the line after waiting   1.03 minutes
Xi at time    7.55, Packets 02 arrived
 at time    8.75, Packets 02 left the line after waiting   1.20 minutes
Xl at time    8.85, Packets 03 arrived
Xv at time    9.20, Packets 04 arrived
 at time    9.87, Packets 03 left the line after waiting   1.02 minutes
 at time   10.41, Packets 04 left the line after waiting   1.22 minutes
Xv at time   12.20, Packets 05 arrived
 at time   13.65, Packets 05 left the line after waiting   1.45 minutes
at time   15.62, Packets 00 finished their rouing at router device 'ethernet'andom_router


### Answer:The Solution is divided into two parts first portion is to get the packtes that are going and also get the connection randomly and then get the number of packet and  simulate the average packet that pass through them and the first portion is pass through 2nd function  source to get the source of the pakcet and the time required to get through

## Problem 2

**Write and discuss the steps to answering the following research question:** Create a function that will automatically run the simulation above given $n$ and return the number of packets that successfully route through. Make sure that the function sets the same random seed inside the function, so that your simulation parameters are consistent across runs. Using `scipy.optimize.minimize_scalar()` ([see reference](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize_scalar.html#scipy.optimize.minimize_scalar)), what is the value of $n$ (number of devices on the network) that results in the highest number of packets that successfully route through our router design, where $5 \le n \le 30$?

In [55]:
def optimzer():
  RANDOM_SEED = 5
  alpha = 6
  gamma = 3
  min_beta = 1
  max_beta = 1.5
  opt_bounds = (5, 30)
  #a,b,c,d,e=source(env, 5, gamma, 12.0, the_rng)
  for i in range(alpha):
    o = scipy.optimize.minimize_scalar(packet, bounds=opt_bounds, args=(env, 'Packets %02d' % i, counter, 12.0, the_rng))
    env.process(o)
    t = the_rng.exponential(interval)
    yield env.timeout(t)

In [57]:
the_rng = np.random.default_rng(RANDOM_SEED)
env = sp.Environment()
# Start processes and run
counter = sp.Resource(env, capacity=1)
env.process(optimzer())
try:
  env.run()
except:
  print("An exception occurred")


Xl at time    0.00, Packets 00 arrived
Xs at time    0.00, Packets 01 arrived
Xs at time    0.00, Packets 02 arrived
Xs at time    0.00, Packets 03 arrived
Xl at time    0.00, Packets 04 arrived
An exception occurred


### Answer: In this problem we can optimze the simultaion of packets and using the fucntion to get the optimze number  of packet pass through and also define the range between 5 to 30 